<a href="https://colab.research.google.com/github/yal521/DSCI441-machine-learning-project/blob/main/DSCI_441_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import numpy as np
from numpy import where
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
from collections import Counter
import sklearn.metrics as metrique
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from keras.backend import cast
from keras.utils import np_utils
from keras.models import Sequential
from keras import backend as K, regularizers, Model, metrics
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.layers import LSTM, Dense, Embedding, Dropout,Input, Attention, Layer, Concatenate, Permute, Dot, Multiply, Flatten


Load and read the data set

In [2]:
# Load the data and read the first few lines of data
Credit_data = pd.read_csv("/content/drive/MyDrive/Yang_DSCI441_Final_Project/dataset1_creditcard.csv")
Credit_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Data Preprocessing

In [3]:
Credit_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

In [4]:
print(Credit_data.isnull().sum())

# this is a pretty clean dataset without missing values

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


Feature Selection

There is another code file using particle swarm optimization to select optimal features. It takes long time to get the results so that I put codes of feature selection into a seperate code file. I will use the results from feature selection code to build the model and assess its performance 

In [5]:
data = pd.read_csv("/content/drive/MyDrive/Yang_DSCI441_Final_Project/dataset1_creditcard.csv", na_filter=True)
col_del = ['Time' ,'V1', 'V2', 'V3','V6', 'V7', 'V8', 'V9','V10','V11','V12','V13','V14','V15','V17','V19','V20','V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28','Amount']

In [6]:
col_keep = data.drop(col_del,axis =1)
col_keep.head()

,V4,V5,V16,V18,Class
0,1.378155,-0.338321,-0.470401,0.025791,0
1,0.448154,0.060018,0.463917,-0.183361,0
2,0.379780,-0.503198,-2.890083,-0.121359,0
3,-0.863291,-0.010309,-1.059647,1.965775,0
4,0.403034,-0.407193,-0.451449,-0.038195,0


In [7]:
col_keep.shape

(284807, 5)

In [8]:
# Dividing the col_keep data into two parts: observation part and label part
TR = col_keep.drop(['Class'], axis = 'columns')
Label = col_keep['Class']

In [9]:
# Generate and plot imbalanced classification dataset
# summarize class distribution
counter_V = Counter(col_keep['Class'])
print(counter_V)
# scatter plot of examples by class label
for label, _ in counter_V.items():
	row = where(col_keep['Class'] == label)[0]

Counter({0: 284315, 1: 492})


In [10]:
# dataset transformation
oversple = SMOTE()
X_rw, y = oversple.fit_resample(TR, col_keep['Class'])
# summarize the new class distribution
counter_V1 = Counter(y)
#print(counter_V1)
# scatter plot of examples by class label
for label, _ in counter_V1.items():
	row1 = where(y == label)[0]

In [11]:
# regularizing the data
X_rw2 = StandardScaler().fit_transform(X_rw)
X_rw2.shape

(568630, 4)